## Cargamos las librerías necesarias para este proceso:

In [ ]:
#Librerias necesarias:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from astropy.stats.lombscargle import LombScargle
from astropy.timeseries import LombScargle
from gatspy import datasets, periodic
from PyAstronomy.modelSuite import KeplerRVModel
from PyAstronomy import funcFit as fuf

%matplotlib inline


## Cargamos la serie de tiempo del sistema a estudiar.

In [ ]:
#Cargamos las series de tiempo:
df1 = pd.read_csv('HD103720_FIDEOS.txt',delim_whitespace=True, header=None)

#Nombramos cada una de las columnas:
headers = ['JD', 'RV', 'RV_ERROR']
df1.columns = headers

#Aquí, debido a que la velocidad radial viene en km/s, multiplicamos *1000 para dejarlo en m/s
df1.RV=df1.RV*1000.
df1.RV_ERROR=df1.RV_ERROR*1000.


#Normalizamos utilizando la media:
df1['RV'] = (df1['RV'] - df1['RV'].mean())
df1['RV_ERROR'] = df1['RV_ERROR'] - df1['RV_ERROR'].mean()

#Para visualizar el dataframe:
print(df1)

#Procedemos a graficar un error bar plot:
plt.errorbar(df1.JD, df1.RV, df1.RV_ERROR, fmt='ok',capsize=3, elinewidth=1)
plt.xlabel('Julian Days (JD)-2400000')
plt.ylabel('Radial Velocity (m/s)')
plt.title('HD 103720', size=12)
plt.grid()
plt.show()


## Aplicamos el periodograma LombScargle para calcular el periodo orbital

In [ ]:
#Cargamos el periodograma LombScargle:
from gatspy.periodic import LombScargleFast

#Aplicamos el modelo a nuestros datos:
model = LombScargleFast().fit(df1.JD, df1.RV, df1.RV_ERROR)
periods, power = model.periodogram_auto(nyquist_factor=100)

#Graficamos 
fig, ax = plt.subplots()
ax.plot(periods, power,'k')
ax.grid()

ax.set(xlim=(1, 6), ylim=(0, 1),
       title= 'Periodograma de HD 103720',
       xlabel='period (days)',
       ylabel='Lomb-Scargle Power');

# A partir de la serie de tiempo, damos un rango estimado donde podría estar el periodo:
model.optimizer.period_range=(1, 5)
P= model.best_period
print("The orbital period for HD 103720 b is %.3f"% P, "days")

#Ponemos la masa de la estrella:
M = 0.794 # Masas solares
P1 = P/365.25 #Periodo en años

#Por la tercera ley de Kepler, calculamos el semieje mayor.
a = ((P1**2)*(M))**(1/3) # Semi eje mayor en unidades astronómicas
print ("El semieje mayor es %.3f"% a, "UA")

## Realizaremos el ajuste a la fase orbital con el paquete KeplerRVModel, con el fin de calcular la semiamplitud de velocidad radial

In [ ]:
# Cargamos los datos que usaremos en el modelo.
jd = np.array((df1.JD)%(P))/(P)
rv = np.array(df1.RV)
# Error
rverr = np.array(df1.RV_ERROR)

# Modelamos un sistema con 1 solo planeta con desplazamiento constante:
krvm = KeplerRVModel(mp=1, deg=0)

# Ponemos la masa de la estrella:
krvm["mstar"] = 0.794


# Damos algunos parámetros iniciales (initial guesses) para los elementos orbitales
krvm["per1"] = 4.556
krvm["K1"] = 90.
krvm["e1"] = 0.1
krvm["tau1"] = 55387.212
krvm["w1"] =262.

# Ajustamos todos los parámetros menos el periodo
krvm.thaw(["K1", "w1", "e1","tau1","c0"])
# ... Ahora también el periodo
krvm.thaw(["per1"])
krvm.fit(jd, rv, yerr=rverr)
#Ahora, obtenemos el mejor modelo ajustado
kmo = krvm.evaluate(jd)

# Procedemos a calcular el chi cuadrado y el RMS
chi = np.sum( (rv - krvm.model)**2 / rverr**2 )
# RMS
rchi = chi / (len(rv) - len(krvm.freeParameters()))
print("chi-square and reduced chi-square: %6.3f, %6.3f" % (chi, rchi))
rms = np.std(rv - krvm.model)
print("RMS: ", rms)

# Tomamos los datos y los organizamos, para realizar el diagrama de fase.
l=np.c_[jd,krvm.model]
f=l[l[:,0].argsort()]

#Procedemos a graficar
plt.title("Ajuste de la fase orbital de Proxima Centauri b")
plt.xlabel("ϕ")
plt.ylabel("Velocidad Radial (m/s)")
plt.errorbar(jd/P, rv, yerr=rverr, fmt='ok',capsize=3, elinewidth=1)
plt.plot(f[:,0]/P, f[:,1], 'r-')
plt.grid()
plt.show()

#Aquí, obtenemos la semiamplitud de velocidad radial.
K = (krvm.model.max()-krvm.model.min())/2
print("La semiamplitud resultante es de: %.3f"% K ,"m/s")

#Calculamos el error de la semiamplitud

n=len(rv)
std=((krvm.model-rv)/n)**2
suma = sum(std)
err=np.sqrt(suma)
err
print("La semiamplitud está dada por: %.2f" % K, "+/- %.2f"% err)

## Usaremos el algoritmo <i>emcee</i> para calcular la excentricidad más probable para este sistema.

In [ ]:
# Datos
jd = np.array(df1.JD)
rv = np.array(df1.RV)
# Error
rverr = np.array(df1.RV_ERROR)


# Modelamos un sistema con 1 solo planeta con desplazamiento constante:
krvm = KeplerRVModel(mp=1, deg=0)

# Ponemos la masa de la estrella:
krvm["mstar"] = 0.794

# Damos algunos parámetros iniciales (initial guesses) para los elementos orbitales
krvm["per1"] = P
krvm["K1"] = K
krvm["e1"] = 0.08
krvm["tau1"] = 2445000.
krvm["w1"] = 250.

# Ajustamos todos los parámetros menos el periodo
krvm.thaw(["K1", "tau1", "w1", "e1", "c0"])
# ... Ahora también el periodo
krvm.thaw(["per1"])
krvm.fit(jd, rv, yerr=rverr)
#Ahora, obtenemos el mejor modelo ajustado
kmo = krvm.evaluate(jd)




# Ahora, hacemos análisis  usando MCMC

# "Quemamos" las primeras 50 iteraciones
# Nos quedamos con las siguientes 50:
sampleArgs = {"iters":50, "burn":50}

# Especificamos un rango en el 
# que esperaríamos encontrar la excentricidad,
# para indicarle al módelo "donde buscar":

priors = {"e1":fuf.FuFPrior("limuniform", upper=0.15, lower=0.0)}

# Iniciamos el muestreo
ps = krvm.fitEMCEE(jd, rv, yerr=rverr, sampleArgs=sampleArgs, scales={"e1":0.01}, dbfile="chain1.emcee", \
    priors=priors)


# Le echamos un vistazo a la parte posterior:
ta2 = fuf.TraceAnalysis("chain1.emcee")


## Posterior a las iteraciones, procedemos a graficar el intervalo de excentricidades más probable, con una certeza del 93 % (3 $\sigma$)

In [ ]:
#  Aquí obtenemos el valor esperado y el intervalo más probable para el valor de la excentricidad:
plt.figure(figsize=(9,6))
ta2.plotHist("e1")
plt.xlabel("Excentricidad")
plt.title("Excentricidad más probable de HD 103720 b")
print("Expectation value for eccentricity: %.3f"% ta2.median("e1"))
print("3 sigma for eccentricity: ", ta2.hpd("e1", cred=0.93))

x2 = np.array(ta2.hpd("e1", cred=0.93))
y2=ta2.median("e1")
mini2=y2-x2[0]
maxi2=x2[1]-y2
print("La excentricidad de la órbita es: %.3f"% y2,"- %.3f "%mini2,"+ %.3f"% maxi2)

## Usamos la fórmula de la masa mínima para calcular la masa mínima del exoplaneta

In [ ]:
# Tomamos la ecuación de la masa mínima en unidades canónicas, obteniendo así:

mpsin =((K)/(28.4329))*((0.83)**(2./3.))*((P/365.)**(1./3.))*(np.sqrt(1-y))
print("La masa mínima es: %.3f"%mpsin, "Mj")

## Calculamos el error en el periodo orbital

In [ ]:

# Llamamos el set de datos:
df1 = pd.read_csv('HD103720_FIDEOS.txt',delim_whitespace=True, header=None)

# Se generan los headers y se asignan a cada columna:
headers = ['JD', 'RV', 'RV_ERROR']
df1.columns = headers

#Normalizamos utilizando la media:
df1['RV'] = df1['RV'] - df1['RV'].mean()
#df1['RV_ERROR'] = df1['RV_ERROR'] - df1['RV_ERROR'].mean()
df1['JD'] = df1['JD']

# Para calcular el error en el periodo, llamamos el siguiente paquete:
from sklearn.model_selection import train_test_split


#Cargamos los datos:
rv=df1.RV
jd=df1.JD

#Uilizamos el 90% de los datos de forma aleatoria para calcular el periodo 10 veces
size=0.1
jd_train, jd_test, rv_train, rv_test, = train_test_split(jd,rv,test_size=size,random_state=1)
jd_train1, jd_test1, rv_train1, rv_test1, = train_test_split(jd,rv,test_size=size,random_state=2)
jd_train2, jd_test2, rv_train2, rv_test2, = train_test_split(jd,rv,test_size=size,random_state=3)
jd_train3, jd_test3, rv_train3, rv_test3, = train_test_split(jd,rv,test_size=size,random_state=4)
jd_train4, jd_test4, rv_train4, rv_test4, = train_test_split(jd,rv,test_size=size,random_state=5)
jd_train5, jd_test5, rv_train5, rv_test5, = train_test_split(jd,rv,test_size=size,random_state=6)
jd_train6, jd_test6, rv_train6, rv_test6, = train_test_split(jd,rv,test_size=size,random_state=7)
jd_train7, jd_test7, rv_train7, rv_test7, = train_test_split(jd,rv,test_size=size,random_state=8)
jd_train8, jd_test8, rv_train8, rv_test8, = train_test_split(jd,rv,test_size=size,random_state=9)
jd_train9, jd_test9, rv_train9, rv_test9, = train_test_split(jd,rv,test_size=size,random_state=10)

# Damos el rango esperado para el periodo orbital:
ran1,ran2 = 2,6
name1="The orbital period for HD 103720 b is "
freq, radV = LombScargle(jd_train,rv_train).autopower()
model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train,rv_train)
P1=model.best_period
print("The orbital period for HD 103720 b is ", P1, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train1,rv_train1)
P2=model.best_period
print("The orbital period for HD 103720 b is ", P2, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train2,rv_train2)
P3=model.best_period
print("The orbital period for HD 103720 b is ", P3, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train3,rv_train3)
P4=model.best_period
print("The orbital period for HD 103720 b is ", P4, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train4,rv_train4)
P5=model.best_period
print("The orbital period for HD 103720 b is ", P5, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train5,rv_train5)
P6=model.best_period
print("The orbital period for HD 103720 b is ", P6, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2)
model.fit(jd_train6,rv_train6)
P7=model.best_period
print("The orbital period for HD 103720 b is ", P7, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train7,rv_train7)
P8=model.best_period
print("The orbital period for HD 103720 b is ", P8, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train8,rv_train8)
P9=model.best_period
print("The orbital period for HD 103720 b is ", P9, "days")

model = periodic.LombScargle(fit_period=True)
model.optimizer.period_range = (ran1,ran2 )
model.fit(jd_train9,rv_train9)
P10=model.best_period
print("The orbital period for HD 103720 b is ", P10, "days")


from scipy import stats


# FInalmente, tomamos los periodos obtenidos, 
# calculamos la media, desviación estándar y obtenemos un error de 3 sigma:
x1=np.array([P1,P2,P3,P5,P6,P7,P8,P9,P10])
print(x1.mean())
print(x1.std())
print(3*stats.sem(x1))

print(name1, "%.5f" % x1.mean(), "+/- %.5f" % x1.std(), "days")

# Calculamos el error en semieje mayor y masa mínima

In [ ]:
#Utilizamos el paquete qexpy para realizar 
#la propagación de errores a los parámetros que 
#calculamos remplazando en sus fórmulas
import numpy as np
import qexpy as q

# HD 103720
M = q.Measurement(0.794, 0.040)
P = q.Measurement(4.55575, 0.00003)
e = q.Measurement(0.082, 0.027)
K = q.Measurement(85.50, 1.74)

# Calculamos error en semieje mayor:
err = (M * ((P / 365.25) ** 2)) ** (1. / 3.)
print("Semimajor axis:", err)

# Calculamos masa mínima y su error:
mpsin = (K / 28.4329) * (M ** (2. / 3.)) * ((P / 365.) ** (1. / 3.)) * (q.sqrt(1 - e**2))
print("Minimum mass:",mpsin)

